<a href="https://colab.research.google.com/github/skylum1/soft_computing/blob/main/Back_Propogation_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Using Back propogation to recognise digits

In [ ]:
from __future__ import print_function
import string
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import random
import numpy as np

In [ ]:
# to convert a font text to pixel matrix
def pixelify(text, path='arialbd.ttf', fontsize=14):

    font = ImageFont.truetype(path, fontsize) 
    w, h = font.getsize(text)  
    h *= 2
    image = Image.new('1', (w, h), 1)  
    draw = ImageDraw.Draw(image)
    draw.text((0, 0), text, font=font) 
    arr = np.asarray(image)
    # print(arr)
    arr = np.where(arr, 0, 1)
    arr = arr[(arr != 0).any(axis=1)]
    return arr

In [ ]:
def display(arr):
    result = np.where(arr==1, '#', ' ')
    
    print('\n'.join([''.join(row) for row in result]))

In [ ]:
#getting font
!gdown --id 115Aaq80dfn06REOJIeaAYEYVflc8NBJ5

In [ ]:
# visualising traing examples
digits='0 1 2'
for i in digits:
  arr = pixelify(i,path='/content/Roboto-Light.ttf',fontsize=10)
  # print(arr)
  display(arr)

  ##  
 #  # 
 #  # 
 #  # 
 #  # 
 #  # 
  ##  

  #   
 ##   
  #   
  #   
  #   
  #   
  #   

  ##  
 #  # 
 #  # 
   #  
  #   
 #    
 #### 


In [ ]:
def generate_training_matrix():
  input =[]
  required_outputs=[]
  digits='0123456789'
  for c in digits:
    arr = pixelify(
        c, 
        path='/content/Roboto-Light.ttf', 
        fontsize=10)
    arr=arr.reshape(-1,)
    input.append(arr)
    i=ord(c)-ord('0')
    ls=np.array([int(dc) for dc in bin(i)[2:]])
    ls=np.where(ls,1,0)
    ls=np.pad(ls,(4-ls.shape[0],0 ), 'constant',constant_values=(0,0))
    required_outputs.append(ls)

  yt=np.asarray(input)
  ls=np.asarray(required_outputs)
  return yt,ls



In [ ]:
def initialize_parameters(ni, nh, no):    
    np.random.seed(1)
    W1 = np.random.randn(nh,ni)*0.01
    b1 = np.zeros((nh,1))
    W2 = np.random.randn(no,nh)*0.01
    b2 = np.zeros((no,1))
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    return parameters  

In [ ]:
inputs,outputs = generate_training_matrix()
number_hidden_units=3
number_input_units=inputs[0].size
number_output_units=outputs[0].size

  

In [ ]:
parameters = initialize_parameters(number_input_units,number_hidden_units,number_output_units)

In [ ]:
def linear_forward(A, W, b):
    Z = np.dot(W,A)+b    
    cache = (A, W, b)
    return Z, cache

In [ ]:
def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    cache = Z 
    return A, cache

In [ ]:
def activation_forward(A_prev, W, b):
    Z, linear_cache = linear_forward(A_prev, W, b)
    A, activation_cache = sigmoid(Z)
    cache = (linear_cache, activation_cache)
    return A, cache

In [ ]:
def model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters)-1
    for l in range(1, L):
        A_prev = A 
        A, cache = activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)])
        caches.append(cache)          
    return A, caches

In [ ]:
 A, cache = activation_forward(inputs[0].reshape(-1,1), parameters['W1'], parameters['b1'])

In [ ]:
def sigmoid_backward(dA, cache):
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    return dZ


In [ ]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = np.dot(dZ,A_prev.T)/m
    db = np.sum(dZ,axis=1,keepdims=True)/m
    dA_prev = np.dot(W.T,dZ)
    return dA_prev, dW, db

In [ ]:
def linear_activation_backward(dA, cache):
    linear_cache, activation_cache = cache
    dZ =  sigmoid_backward(dA, activation_cache)
    dA_prev, dW, db =  linear_backward(dZ, linear_cache)
    return dA_prev, dW, db

In [ ]:
def model_backward(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) 
    dAL=Y-AL
    current_cache = caches[L-1]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dAL,current_cache)
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] =dW_temp 
    grads["db" + str(L)] =db_temp
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l+1)],current_cache )
        grads["dA" + str(l)] =dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] =db_temp
    return grads

In [ ]:

def update_parameters(params, grads, learning_rate):
    parameters = params.copy()
    L = len(parameters)-2
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)]+learning_rate* grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] =parameters["b" + str(l+1)]+learning_rate* grads["db"+str(l+1)]
    return parameters

In [ ]:
# trainning 
for i in range(0,1000):
  for j in range(0,10):
    y,caches=model_forward(inputs[j].reshape(-1,1),parameters)
    grads=model_backward(y,outputs[j],caches)
    parameters=update_parameters(parameters,grads,0.5)


In [ ]:
def activation(ar,theta=0.2):
  for i in range(ar.size): 
    if ar[i]>=theta:
       ar[i]= 1
    else: 
     ar[i]= 0;  
  return ar


In [ ]:
def array_to_int(arr):
  b=np.where(arr==1,1,0)
  ans=b.dot(1 << np.arange(b.size)[::-1])
  return ans
    

In [ ]:
 ## Validation 
 for i in range(0,10):
    y,caches=model_forward(inputs[i].reshape(-1,1),parameters)
    mul=activation(y.squeeze(),theta=0.5)
    print('Input: ' +str(i),'prediction:'+ str(array_to_int(mul)))

Input: 0 prediction:0
Input: 1 prediction:1
Input: 2 prediction:2
Input: 3 prediction:3
Input: 4 prediction:4
Input: 5 prediction:5
Input: 6 prediction:6
Input: 7 prediction:7
Input: 8 prediction:8
Input: 9 prediction:9


In [ ]:
 # outputs before activation
 for j in range(0,10):
    y,caches=model_forward(inputs[j].reshape(-1,1),parameters)
    print(y.squeeze())
    print(outputs[j])


[0.09230073 0.09241207 0.04989085 0.02242182]
[0 0 0 0]
[0.06908304 0.0690052  0.0341838  0.99882652]
[0 0 0 1]
[0.02974469 0.06609642 0.95728585 0.01138124]
[0 0 1 0]
[0.01680511 0.06320134 0.94915944 0.99345213]
[0 0 1 1]
[1.04098074e-05 9.98600000e-01 4.43033121e-02 1.55382069e-03]
[0 1 0 0]
[1.77238626e-06 9.99350848e-01 1.09091135e-01 9.56869896e-01]
[0 1 0 1]
[2.51323060e-04 8.84363926e-01 8.85607456e-01 5.13481094e-02]
[0 1 1 0]
[6.23918146e-05 9.36281763e-01 9.36122786e-01 9.87690886e-01]
[0 1 1 1]
[9.53790144e-01 7.02065043e-04 3.66936070e-02 5.32361892e-02]
[1 0 0 0]
[0.88313063 0.00107717 0.05876095 0.94711992]
[1 0 0 1]
